<a href="https://colab.research.google.com/github/chiquynhdang03/Dang-Quynh-Chi/blob/main/gg_sheet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# --- Initial Environment Setup (MUST BE AT THE TOP) ---
COLAB_ENV = False
try:
    # Attempt to import google.colab to detect Colab environment
    from google.colab import drive
    COLAB_ENV = True
except ImportError:
    pass # Not in Colab environment

# --- Install Dependencies (For Colab, run this cell first) ---
# In Colab, you'd typically run this in a separate cell.
# If running in GitHub Actions, these are handled by the workflow's 'Install dependencies' step.
if COLAB_ENV:
    print("Installing/Upgrading Python dependencies in Colab...")
    #!pip install pandas requests google-auth google-api-python-client urllib3 openpyxl --upgrade
    print("Dependencies installed.")
else:
    print("Skipping pip install, dependencies handled by GitHub Actions environment.")

#from google.colab import drive
#drive.mount('/content/drive')

import pandas as pd
import requests
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
import os
from datetime import datetime, timedelta
import time # Import time for sleep functionality
import io # Import io for string stream stream handling
import ssl # NEW IMPORT

# NEW IMPORTS for direct Google API client authentication
from google.oauth2 import service_account
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseUpload, MediaIoBaseDownload
from googleapiclient.errors import HttpError # Import HttpError for specific error handling

# --- Configuration Constants ---
# Changed to CSV endpoint
NYC_OPEN_DATA_RESOURCE_URL = "https://data.cityofnewyork.us/resource/erm2-nwe9.csv"
API_LIMIT_PER_REQUEST = 1000
ZIP_CODES_TO_INCLUDE = [10004, 10005, 10006, 10007, 10038, 10280, 10282, 10013, 10002]

# --- Google Sheet File Configuration ---
# Name for the Google Sheet file in Drive (it will be a native Google Sheet)
GSHEET_FILE_NAME = "CB1_311_Complaint_Data_GSheet" # No .xlsx extension needed for native Google Sheet
GSHEET_MIMETYPE = "application/vnd.google-apps.spreadsheet"
# This is the name of the file as it will appear in Google Drive
GSHEET_DRIVE_FILE_NAME = "CB1_311_Complaint_Data_GSheet" # Renamed for clarity

# --- Google Drive Configuration ---
# IMPORTANT: Replace with the actual ID of your Google Drive folder
# where you want to store the Google Sheet file.
# To get this: Go to your Google Drive, create a new folder, open it,
# the ID is in the URL: https://drive.google.com/drive/folders/YOUR_ID
GOOGLE_DRIVE_FOLDER_ID = "0AI4Egw2Y1IwhUk9PVA" # <--- UPDATED

# Service Account Key file name (downloaded from GCP)
SERVICE_ACCOUNT_KEY_FILE = "/content/drive/Shareddrives/311_Complaint_Data/service_account_key.json"

# 3. DEFINITIVE FIX: Set the start date to the PAST
DEFAULT_INITIAL_FETCH_DATE = "2025-04-01"

# --- Checkpointing Functions ---
# For Colab, progress file should also be in Drive or a persistent location
PROGRESS_FILE_NAME = "last_processed_date.txt" # Example for Colab persistent storage
PROGRESS_FILE_MIMETYPE = "text/plain"

def read_last_processed_date(drive_service) -> str:
    """Reads the last processed date from the progress file (in Google Drive)."""
    print("Attempting to read last processed date from Google Drive...")
    try:
        # Search for the progress file within the specified GOOGLE_DRIVE_FOLDER_ID
        file_list_response = drive_service.files().list(
            q=f"'{GOOGLE_DRIVE_FOLDER_ID}' in parents and trashed=false and name='{PROGRESS_FILE_NAME}'",
            spaces='drive',
            fields='files(id, name)'
        ).execute()
        file_items = file_list_response.get('files', [])


        if file_items:
            progress_file_id = file_items[0]['id']
            print(f"Found progress file in Drive (ID: {progress_file_id}). Downloading...")
            request = drive_service.files().get_media(fileId=progress_file_id)
            file_content_bytes = io.BytesIO()
            downloader = MediaIoBaseDownload(file_content_bytes, request)
            done = False
            while done is False:
                status, done = downloader.next_chunk()
                if status:
                    print(f"Download {int(status.progress() * 100)}%.")
            file_content_bytes.seek(0)

            date_str = file_content_bytes.getvalue().decode('utf-8').strip()
            try:
                datetime.strptime(date_str, '%Y-%m-%d')
                print(f"Resuming from last processed date: {date_str}")
                return date_str
            except ValueError:
                print(f"Invalid date format in Drive progress file. Starting from default.")
                return DEFAULT_INITIAL_FETCH_DATE
        print(f"No progress file found in Drive. Starting from default date: {DEFAULT_INITIAL_FETCH_DATE}")
        return DEFAULT_INITIAL_FETCH_DATE
    except Exception as e:
        print(f"Error reading progress file from Drive: {e}. Starting from default.")
        return DEFAULT_INITIAL_FETCH_DATE


def write_last_processed_date(date_str: str, drive_service):
    """Writes the last successfully processed date to the progress file (in Google Drive)."""
    try:
        progress_buffer = io.BytesIO(date_str.encode('utf-8'))

        # Search for the progress file within the specified GOOGLE_DRIVE_FOLDER_ID
        file_list_response = drive_service.files().list(
            q=f"'{GOOGLE_DRIVE_FOLDER_ID}' in parents and trashed=false and name='{PROGRESS_FILE_NAME}'",
            spaces='drive',
            fields='files(id, name)'
        ).execute()
        file_items = file_list_response.get('files', [])


        if file_items:
            progress_file_id = file_items[0]['id']
            media = MediaIoBaseUpload(progress_buffer, mimetype=PROGRESS_FILE_MIMETYPE, resumable=True)
            drive_service.files().update(
                fileId=progress_file_id,
                media_body=media,
                fields='id, name'
            ).execute()
            print(f"Progress saved to Drive: Last processed date is now {date_str}")
        else:
            # Create new file in the specified GOOGLE_DRIVE_FOLDER_ID
            file_metadata = {
                'name': PROGRESS_FILE_NAME,
                'parents': [GOOGLE_DRIVE_FOLDER_ID], # Set parent folder
                'mimeType': PROGRESS_FILE_MIMETYPE
            }
            media = MediaIoBaseUpload(progress_buffer, mimetype=PROGRESS_FILE_MIMETYPE, resumable=True)
            drive_service.files().create(
                body=file_metadata,
                media_body=media,
                supportsAllDrives=True, # Crucial for Shared Drives
                fields='id, name'
            ).execute()
            print(f"Progress file created and saved to Drive: Last processed date is now {date_str}")
    except Exception as e:
        print(f"Warning: Could not save progress to Drive file: {e}")

# --- Helper Function to Fetch Data from NYC Open Data API with Pagination ---
def fetch_nyc_data_incremental(start_date_str: str, end_date_str: str = None) -> pd.DataFrame:
    """
    Fetches NYC 311 data. This version uses a simple date filter that is more reliable.
    """
    all_fetched_dfs = []
    offset = 0
    more_data_available = True

    # Build a simple WHERE clause for the date filter
    if end_date_str:
        where_clause = f"created_date >= '{start_date_str}T00:00:00.000' AND created_date <= '{end_date_str}T23:59:59.999'"
        print(f"Starting API fetch from {start_date_str} to {end_date_str}...")
    else:
        where_clause = f"created_date > '{start_date_str}T23:59:59.999'"
        print(f"Starting API fetch from {start_date_str} to present...")

    session = requests.Session()
    retries = Retry(total=5, backoff_factor=1, status_forcelist=[500, 502, 503, 504])
    session.mount('https://', HTTPAdapter(max_retries=retries))

    while more_data_available:
        # Use simple URL parameters instead of a complex $query
        params = {
            '$limit': API_LIMIT_PER_REQUEST,
            '$offset': offset,
            '$where': where_clause,
            '$order': 'created_date ASC'
        }

        try:
            print(f"Fetching page with offset: {offset}...")
            response = session.get(NYC_OPEN_DATA_RESOURCE_URL, params=params, timeout=60)
            response.raise_for_status()

            # Check for empty response text before reading CSV
            if not response.text.strip():
                print("API returned an empty response. Ending fetch.")
                more_data_available = False
                continue

            page_df = pd.read_csv(io.StringIO(response.text))

            if not page_df.empty:
                all_fetched_dfs.append(page_df)
                offset += len(page_df) # Increment offset by records received
                if len(page_df) < API_LIMIT_PER_REQUEST:
                    more_data_available = False # Last page
            else:
                more_data_available = False

        except requests.exceptions.RequestException as e:
            print(f"Error fetching data from API: {e}")
            break # Exit loop on error
        except pd.errors.EmptyDataError:
            print("No more data to process (Empty CSV).")
            more_data_available = False

    if not all_fetched_dfs:
        print("No new records found from API.")
        return pd.DataFrame()

    df = pd.concat(all_fetched_dfs, ignore_index=True)
    print(f"Total records fetched from API (before filtering): {len(df)}")
    return df

# --- Main Function to Update Google Sheet and Upload to Google Drive ---
def update_google_sheet_data():
    """
    Main function to update the Google Sheet file with new data and upload it to Google Drive.
    This function handles both initial historical load (chunked by year) and daily incremental updates.
    """
    print("--- Starting Google Sheet Update and Google Drive Upload Process (Pure In-Memory) ---")


    # --- Step 0: Authenticate with Google Drive (Service Account) ---
    print("Authenticating with Google Drive...")
    try:
        # Mount Drive if in Colab (only if not already mounted)
        if COLAB_ENV and not os.path.exists('/content/drive'):
            drive.mount('/content/drive')


        creds = service_account.Credentials.from_service_account_file(
            SERVICE_ACCOUNT_KEY_FILE,
            scopes=['https://www.googleapis.com/auth/drive.file']
        )
        drive_service = build('drive', 'v3', credentials=creds)
        print("Google Drive authentication successful.")
    except Exception as e:
        print(f"Google Drive authentication failed: {e}")
        print(f"Please ensure '{SERVICE_ACCOUNT_KEY_FILE}' is in the correct Google Drive folder and accessible via mounting.")
        print(f"Also check Service Account permissions in Google Cloud Console and Drive folder sharing.")
        return # Exit if authentication fails


    # --- Step 1: Download existing Google Sheet from Google Drive (if exists) ---
    existing_df = pd.DataFrame()
    last_created_date_in_drive = None

    # Search for the file in the specified folder
    file_id_in_drive = None
    try:
        file_list_response = drive_service.files().list(
            q=f"'{GOOGLE_DRIVE_FOLDER_ID}' in parents and trashed=false and name='{GSHEET_DRIVE_FILE_NAME}'",
            spaces='drive',
            fields='files(id, name, mimeType)' # Request mimeType to check if it's a native GSheet
        ).execute()
        file_items = file_list_response.get('files', [])


        if file_items:
            file_id_in_drive = file_items[0]['id']
            print(f"Found existing Google Sheet (ID: {file_id_in_drive}). Downloading...")

            # Download the file content into a BytesIO object
            # For native Google Sheets, you must export them as CSV to read with pandas
            if file_items[0]['mimeType'] == GSHEET_MIMETYPE:
                request = drive_service.files().export_media(fileId=file_id_in_drive, mimeType='text/csv')
            else: # If it's a regular CSV file (e.g., uploaded directly)
                request = drive_service.files().get_media(fileId=file_id_in_drive)


            file_content_bytes = io.BytesIO()
            downloader = MediaIoBaseDownload(file_content_bytes, request)
            done = False
            while done is False:
                status, done = downloader.next_chunk()
                if status:
                    print(f"Download {int(status.progress() * 100)}%.")
            file_content_bytes.seek(0)

            try:
                # Read the BytesIO object directly into a DataFrame
                existing_df = pd.read_csv(file_content_bytes)
                if 'created_date' in existing_df.columns and not existing_df.empty:
                    existing_df['created_date'] = pd.to_datetime(existing_df['created_date'], errors='coerce')
                    last_created_date_in_drive = existing_df['created_date'].max()
                    print(f"Latest created_date found in Google Drive Google Sheet: {last_created_date_in_drive}")
                else:
                    print("Google Drive Google Sheet is empty or 'created_date' column is missing. Starting from default date.")
            except pd.errors.EmptyDataError:
                print(f"Google Drive Google Sheet '{GSHEET_DRIVE_FILE_NAME}' is empty. Will create a new one.")
                existing_df = pd.DataFrame()
            except Exception as e:
                print(f"Error reading existing Google Sheet from Google Drive: {e}. Starting with empty DataFrame.")
                existing_df = pd.DataFrame()
        else:
            print("Google Sheet not found in Google Drive. Will create a new one.")
    except HttpError as error:
        print(f"An HTTP error occurred while checking/downloading from Drive: {error}")
        print("Assuming no existing file in Drive for now.")
        file_id_in_drive = None
    except Exception as e:
        print(f"An unexpected error occurred during Drive file check/download: {e}")
        print("Assuming no existing file in Drive for now.")
        file_id_in_drive = None




    # Determine the start date for fetching new data
    if last_created_date_in_drive: # Use date from Drive if available
        fetch_start_date = (last_created_date_in_drive + timedelta(days=1)).strftime('%Y-%m-%d')
    else: # Otherwise, use checkpoint or default
        last_processed_date_str = read_last_processed_date(drive_service) # Pass drive_service
        fetch_start_date = (datetime.strptime(last_processed_date_str, '%Y-%m-%d') + timedelta(days=1)).strftime('%Y-%m-%d')

    # If fetch_start_date is before DEFAULT_INITIAL_FETCH_DATE, clamp it
    if datetime.strptime(fetch_start_date, '%Y-%m-%d') < datetime.strptime(DEFAULT_INITIAL_FETCH_DATE, '%Y-%m-%d'):
        fetch_start_date = DEFAULT_INITIAL_FETCH_DATE
        print(f"Adjusted fetch start date to DEFAULT_INITIAL_FETCH_DATE: {fetch_start_date}")




    print(f"Determining data fetch strategy. Actual fetch start date: {fetch_start_date}")


    # --- Step 3: Fetch Data (Historical Chunks or Incremental) ---
    all_fetched_data_frames = []


    # This logic now primarily handles the initial full load or large gaps
    # If the Google Sheet is empty in Drive OR the last processed date is significantly old (e.g., from a previous year)
    # OR if the checkpoint date is very old, perform historical chunking.
    current_year = datetime.now().year
    start_year_for_chunking = datetime.strptime(fetch_start_date, '%Y-%m-%d').year


    # Only perform chunking if we're starting from a historical year (not just a few days ago in current year)
    # And if we haven't completed the historical load yet.
    # The condition `not existing_df.empty` here is crucial: if existing_df is empty, it means Drive file was not found or empty
    # so we need to do a full historical pull.
    if not existing_df.empty and start_year_for_chunking < current_year: # Only chunk if existing_df is NOT empty and we are in a historical year
        print("Performing historical data fetch in yearly chunks...")
        for year in range(start_year_for_chunking, current_year + 1):
            year_start_date = f"{year}-01-01"
            year_end_date = f"{year}-12-31"

            if year == current_year:
                year_end_date = datetime.now().strftime('%Y-%m-%d')
                if datetime.strptime(year_start_date, '%Y-%m-%d') > datetime.strptime(year_end_date, '%Y-%m-%d'):
                    continue


            print(f"\n--- Fetching data for year: {year} ({year_start_date} to {year_end_date}) ---")

            # Only fetch if the year's start date is relevant to the overall fetch_start_date
            if datetime.strptime(year_end_date, '%Y-%m-%d') >= datetime.strptime(fetch_start_date, '%Y-%m-%d'):
                # Adjust year_start_date if it's earlier than fetch_start_date (for first relevant chunk)
                actual_chunk_start_date = max(datetime.strptime(year_start_date, '%Y-%m-%d'), datetime.strptime(fetch_start_date, '%Y-%m-%d')).strftime('%Y-%m-%d')

                chunk_df = fetch_nyc_data_incremental(actual_chunk_start_date, year_end_date)
                if not chunk_df.empty:
                    all_fetched_data_frames.append(chunk_df)
                else:
                    print(f"No data found for year {year} or fetch failed.")
            else:
                print(f"Skipping year {year} as it's older than the required fetch_start_date ({fetch_start_date}).")


        if not all_fetched_data_frames:
            print("No new data fetched during historical chunking. Google Sheet will not be updated.")
            return

        new_data_df = pd.concat(all_fetched_data_frames, ignore_index=True)
        print(f"Total records fetched during historical chunking: {len(new_data_df)}")


    else:
        print(f"Performing daily incremental data fetch from: {fetch_start_date}...")
        new_data_df = fetch_nyc_data_incremental(fetch_start_date)

        if not new_data_df.empty:
            print("Filtering data in Python...")

            # 1. Filter for the correct community board
            # The .str.contains() is a flexible way to find the board
            filtered_df = new_data_df[new_data_df['community_board'].str.contains('01 MANHATTAN', na=False)].copy()
            print(f"Rows after community board filter: {len(filtered_df)}")

            # 2. Filter for the correct zip codes
            # The .isin() method is the Pandas equivalent of SQL's IN
            filtered_df = filtered_df[filtered_df['incident_zip'].isin(ZIP_CODES_TO_INCLUDE)].copy()
            print(f"Rows after zip code filter: {len(filtered_df)}")

            new_data_df = filtered_df # Assign the final filtered data
        if new_data_df.empty:
            print("No new data fetched from API for incremental update. Google Sheet will not be updated.")
            return


    # --- Step 3: Combine existing data with new data ---
    # Ensure column order is consistent before concat
    if not existing_df.empty:
        new_data_df = new_data_df.reindex(columns=existing_df.columns, fill_value=None)
        combined_df = pd.concat([existing_df, new_data_df], ignore_index=True)
    else:
        combined_df = new_data_df

    # Optional: Remove potential duplicates if 'unique_key' exists
    if 'unique_key' in combined_df.columns:
        initial_rows = len(combined_df)
        combined_df.drop_duplicates(subset=['unique_key'], inplace=True, keep='last')
        if len(combined_df) < initial_rows:
            print(f"Removed {initial_rows - len(combined_df)} duplicate rows based on 'unique_key'.")


    print(f"Total rows in combined DataFrame: {len(combined_df)}")


    # --- Step 4: Prepare CSV content in memory for upload ---
    csv_buffer = io.StringIO()
    # Write to CSV in-memory, then encode to bytes
    combined_df.to_csv(csv_buffer, index=False, encoding='utf-8')
    csv_buffer.seek(0) # Rewind to the beginning of the buffer


    # --- Step 5: Upload/Update Google Sheet file to Google Drive ---
    print("Uploading/Updating Google Sheet file to Google Drive...")

    # Use file_id_in_drive to determine if update or create
    if file_id_in_drive: # File was found and downloaded in Step 1
        # Update existing file
        file_metadata = {'name': GSHEET_FILE_NAME} # Name is sufficient for update

        # Use MediaIoBaseUpload for in-memory content (CSV bytes)
        media = MediaIoBaseUpload(
            io.BytesIO(csv_buffer.getvalue().encode('utf-8')), # Pass the BytesIO object directly
            mimetype='text/csv', # Source MIME type is CSV
            resumable=True
        )


        try: # Added try-except for the update operation
            updated_file = drive_service.files().update(
                fileId=file_id_in_drive, # Use the ID of the found file
                body=file_metadata, # Pass file_metadata as body
                media_body=media,
                supportsAllDrives=True, # Crucial for Shared Drives
                fields='id, name'
            ).execute()
            print(f"Updated existing Google Sheet on Google Drive (ID: {updated_file.get('id')}).")
            # Write checkpoint after successful upload
            write_last_processed_date(datetime.now().strftime('%Y-%m-%d'), drive_service)
        except HttpError as update_error:
            if update_error.resp.status == 404:
                print(f"Warning: Google Sheet with ID {file_id_in_drive} not found during update. Attempting to create a new one instead.")
                file_id_in_drive = None # Reset ID so it falls into the 'else' block for creation
            else:
                raise # Re-raise other HttpErrors


    if not file_id_in_drive: # If file was not found initially, or 404 on update
        # Upload new file (as a native Google Sheet)
        file_metadata = {
            'name': GSHEET_FILE_NAME,
            'parents': [GOOGLE_DRIVE_FOLDER_ID],
            'mimeType': GSHEET_MIMETYPE # Set MIME type for native Google Sheet
        }

        # Use MediaIoBaseUpload for in-memory content (CSV bytes)
        media = MediaIoBaseUpload(
            io.BytesIO(csv_buffer.getvalue().encode('utf-8')), # Pass the BytesIO object directly
            mimetype='text/csv', # Source MIME type is CSV
            resumable=True
        )

        # Create a new file, Google Drive will convert the CSV to a native Google Sheet
        new_file = drive_service.files().create(
            body=file_metadata,
            media_body=media,
            supportsAllDrives=True, # Crucial for Shared Drives
            fields='id, name'
        ).execute()
        print(f"Uploaded new Google Sheet to Google Drive (ID: {new_file.get('id')}).")
        # Write checkpoint after successful upload (for new file creation)
        write_last_processed_date(datetime.now().strftime('%Y-%m-%d'), drive_service)


    print("--- Google Sheet Update and Google Drive Upload Process Completed ---\n")


if __name__ == "__main__":
    update_google_sheet_data()


Skipping pip install, dependencies handled by GitHub Actions environment.


--- Starting Google Sheet Update and Google Drive Upload Process (Pure In-Memory) ---
Authenticating with Google Drive...
Google Drive authentication failed: [Errno 2] No such file or directory: '/content/drive/Shareddrives/311_Complaint_Data/service_account_key.json'
Please ensure '/content/drive/Shareddrives/311_Complaint_Data/service_account_key.json' is in the correct Google Drive folder and accessible via mounting.
Also check Service Account permissions in Google Cloud Console and Drive folder sharing.
